## MLJAR

### Instalacja

https://github.com/mljar/mljar-supervised

In [ ]:
!pip install mljar-supervised

In [ ]:
# przykład wywołania mljar

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from supervised.automl import AutoML # mljar-supervised

# Load the data
housing = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(
    pd.DataFrame(housing.data, columns=housing.feature_names),
    housing.target,
    test_size=0.25,
    random_state=123,
)

# train models with AutoML
automl = AutoML(mode="Explain")
automl.fit(X_train, y_train)

# compute the MSE on test data
predictions = automl.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, predictions))

### Zadanie 1

Wybierz zbiór danych - może być to na przykład zbiór danych, którym pracowałeś/aś podczas laboratorium z `autogluon`. Przygotuj model/e używając pakietu `mljar`.

Bazując na wiedzy o przygotowanych modelach i ich jakości porównaj jak poradził sobie pakiet z tym zadaniem. Odpowiedz na pytania:

a) jakie typy modeli rozważył framework?

b) jaką miarę wylicza jako bazową?

Bazująć na wiedz jak działa `autogluon`. Porównaj te dwa frameworki i odpowiedz na pytania:

c) który uzyskał lepsze rezultaty?

d) czy w obu przypadkach komitet modeli równie dobrze się sprawdził? (tzn. czy komitet modeli jest najlepszym modelem jaki uzyskaliśmy)


### Zadanie 2

Porównaj wyniki uzyskane różnymi trybami (*mode*) pracy pakietu `mljar`. Czy otrzymane wyniki są znacząco różne. Czy wykorzystując podstawowy tryb jesteśmy w stanie uzyskać dobre wyniki?

In [ ]:
# train models with AutoML
automl = AutoML(mode="Perform", total_time_limit = 60)
automl.fit(X_train, y_train)

### Zadanie 3

Fairness Aware Training (more https://mljar.com/blog/fairness-machine-learning/)

- `fairness_metric` - metric which will be used to decide if the model is fair,

- `fairness_threshold` - threshold used in decision about model fairness,

- `privileged_groups` privileged groups used in fairness metrics computation,

- `underprivileged_groups` - underprivileged groups used in fairness metrics computation.


The `fit()` method accepts `sensitive_features`. When sensitive features are passed to AutoML, the best model will be selected among fair models only. In the AutoML reports, additional information about fairness metrics will be added. The MLJAR AutoML supports two methods for bias mitigation:

- Sample Weighting - assigns weights to samples to treat samples equally,
- Smart Grid Search - similar to Sample Weighting, where different weights are checked to optimize fairness metric.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from supervised.automl import AutoML

data = fetch_openml(data_id=1590, as_frame=True)
X = data.data
y = (data.target == ">50K") * 1
sensitive_features = X[["sex"]]

X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(
    X, y, sensitive_features, stratify=y, test_size=0.75, random_state=42
)

automl = AutoML(
    algorithms=[
        "Xgboost"
    ],
    train_ensemble=False,
    fairness_metric="demographic_parity_ratio",
    fairness_threshold=0.8,
    privileged_groups = [{"sex": "Male"}],
    underprivileged_groups = [{"sex": "Female"}],
)

automl.fit(X_train, y_train, sensitive_features=S_train)